In [21]:
# keller_clique_always_print.py
import pickle
import random
import time
from typing import List

class KellerCliqueSimple:
    """
    Simple version that always prints the clique vertices.
    """
    
    MAX_CLIQUES = {3: 5, 4: 12, 5: 28, 6: 60, 7: 124}
    
    def __init__(self, dimension: int):
        self.dimension = dimension
        self.MAX_CLIQUE_SIZE = self.MAX_CLIQUES[dimension]
        
        # Load codes
        filename = f'keller_codes_d{dimension}.data'
        with open(filename, 'rb') as f:
            self.codes_int = pickle.load(f)
            
        self.n = len(self.codes_int)
        print(f"K({dimension}): {self.n:,} vertices, target clique: {self.MAX_CLIQUE_SIZE}")

    def get_pair_conflict(self, u: int, v: int) -> int:
        return 1 if (self.codes_int[u] & self.codes_int[v]) == 0 else 0

    def compute_conflict_score(self, vertices: List[int]) -> int:
        if len(vertices) < 2: 
            return 0
        total = 0
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                total += self.get_pair_conflict(vertices[i], vertices[j])
        return total

    def run(self, num_restarts=200):
        start_time = time.perf_counter()
        best_clique = []
        best_size = 0
        
        for restart in range(num_restarts):
            if best_size == self.MAX_CLIQUE_SIZE:
                break
            
            # Try random start
            start_size = random.randint(
                max(1, self.MAX_CLIQUE_SIZE - 5),
                min(self.MAX_CLIQUE_SIZE, self.n)
            )
            current = random.sample(range(self.n), start_size)
            
            # Simple hill climbing (simplified)
            for _ in range(100):
                conflicts = self.compute_conflict_score(current)
                if conflicts == 0:
                    break
                
                # Remove worst vertex
                worst_idx = -1
                worst_score = -1
                for i, v in enumerate(current):
                    temp = current[:i] + current[i+1:]
                    score = sum(self.get_pair_conflict(v, u) for u in temp)
                    if score > worst_score:
                        worst_score = score
                        worst_idx = i
                
                if worst_idx == -1:
                    break
                    
                # Try to find better replacement
                removed = current.pop(worst_idx)
                best_new = -1
                best_conflicts = float('inf')
                
                for v in range(self.n):
                    if v in current:
                        continue
                    new_conflicts = sum(self.get_pair_conflict(v, u) for u in current)
                    if new_conflicts < best_conflicts:
                        best_conflicts = new_conflicts
                        best_new = v
                        if best_conflicts == 0:
                            break
                
                if best_new != -1:
                    current.append(best_new)
            
            # Expand if valid
            if self.compute_conflict_score(current) == 0:
                # Greedy expansion
                expanded = list(current)
                added = True
                while added and len(expanded) < self.MAX_CLIQUE_SIZE:
                    added = False
                    for v in range(self.n):
                        if v in expanded:
                            continue
                        if all(self.get_pair_conflict(v, u) == 0 for u in expanded):
                            expanded.append(v)
                            added = True
                            break
                
                if len(expanded) > best_size:
                    best_size = len(expanded)
                    best_clique = expanded.copy()
                    print(f"  Found clique of size {best_size}")
        
        end_time = time.perf_counter()
        runtime_ms = (end_time - start_time) * 1000
        
        print(f"\nResult for K({self.dimension}):")
        print(f"  Size: {best_size}/{self.MAX_CLIQUE_SIZE}")
        print(f"  Runtime: {runtime_ms:.2f} ms")
        print(f"  Vertices ({len(best_clique)}):")
        
        # Print in a readable format
        sorted_clique = sorted(best_clique)
        print("  [", end="")
        for i, v in enumerate(sorted_clique):
            print(f"{v}", end="")
            if i < len(sorted_clique) - 1:
                print(", ", end="")
            if (i + 1) % 10 == 0 and i < len(sorted_clique) - 1:
                print("\n   ", end="")
        print("]")
        
        return sorted_clique, runtime_ms


# Run for specific dimensions
if __name__ == "__main__":
    for d in [3,4,5,6, 7]:  # Just test K(6) and K(7)
        print(f"\n{'='*60}")
        print(f"Keller K({d})")
        print(f"{'='*60}")
        solver = KellerCliqueSimple(d)
        clique, runtime = solver.run(num_restarts=300 if d == 6 else 500)


Keller K(3)
K(3): 64 vertices, target clique: 5
  Found clique of size 5

Result for K(3):
  Size: 5/5
  Runtime: 0.12 ms
  Vertices (5):
  [0, 1, 2, 17, 42]

Keller K(4)
K(4): 256 vertices, target clique: 12
  Found clique of size 12

Result for K(4):
  Size: 12/12
  Runtime: 0.11 ms
  Vertices (12):
  [0, 1, 2, 55, 82, 86, 96, 132, 152, 185, 
   223, 237]

Keller K(5)
K(5): 1,024 vertices, target clique: 28
  Found clique of size 28

Result for K(5):
  Size: 28/28
  Runtime: 0.38 ms
  Vertices (28):
  [0, 1, 62, 89, 113, 118, 162, 190, 288, 289, 
   367, 378, 410, 510, 585, 674, 692, 735, 770, 785, 
   821, 863, 892, 920, 939, 948, 1011, 1013]

Keller K(6)
K(6): 4,096 vertices, target clique: 60
  Found clique of size 60

Result for K(6):
  Size: 60/60
  Runtime: 2.00 ms
  Vertices (60):
  [0, 1, 2, 3, 29, 216, 304, 540, 637, 694, 
   735, 763, 811, 841, 931, 939, 997, 1140, 1203, 1232, 
   1351, 1375, 1377, 1603, 1915, 1968, 1971, 1996, 2012, 2138, 
   2166, 2237, 2264, 2272, 2299,

In [19]:
solver = KellerCliqueGeneral(7)
clique, runtime = solver.run_hill_climbing()
    
print(f"\nTotal runtime: {runtime:.2f} ms")

Loaded K(7) with 16384 vertices.
Target clique size: 124
Searching for clique of size 124 (using 500 restarts)
  Restart 1: Found clique of size 124

FINAL RESULT for K(7)

✓ Clique found:
  Size: 124
  Target: 124
  ✅ SUCCESS: Found maximum clique!
  Runtime: 50.83 ms

Total runtime: 50.83 ms


In [20]:
clique

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 36,
 225,
 356,
 577,
 861,
 921,
 1301,
 1349,
 1377,
 1465,
 1646,
 1668,
 1677,
 1827,
 1852,
 1859,
 2154,
 2416,
 2533,
 2539,
 2778,
 2793,
 2844,
 2848,
 2871,
 3229,
 3616,
 3754,
 3760,
 3836,
 4110,
 4510,
 4631,
 4650,
 4759,
 4780,
 4826,
 4830,
 5054,
 5318,
 5358,
 5500,
 5533,
 5764,
 6118,
 6272,
 6291,
 6473,
 6538,
 6704,
 6755,
 6943,
 7090,
 7364,
 7410,
 7862,
 8411,
 8529,
 8555,
 8608,
 8642,
 8723,
 8724,
 8751,
 8862,
 9182,
 9213,
 9272,
 9638,
 9695,
 9711,
 9904,
 10106,
 10135,
 10304,
 10321,
 10354,
 10433,
 10976,
 11031,
 11475,
 11730,
 12046,
 12330,
 12410,
 12448,
 12612,
 12805,
 13145,
 13286,
 13338,
 13795,
 13860,
 13963,
 14082,
 14126,
 14195,
 14313,
 14333,
 14363,
 14445,
 14724,
 14754,
 14839,
 14906,
 14930,
 15182,
 15445,
 15465,
 15513,
 15594,
 15848,
 15862,
 15901,
 16089,
 16300]